In [1]:
import numpy as np
from scipy import misc as misc
import tensorflow as tf
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
tf.set_random_seed(777)
tmp = np.load('/data1/jerry/face/Network/data2.npz')
train_img = tmp['train_img']
train_lab = tmp['train_lab']


In [3]:
new = np.zeros([232,480,480,3])

new[:,128:352, 128:352, :] = train_img

In [4]:
num_classes = 2
X = tf.placeholder(tf.float32, shape=[None, 480,480,3])
Y = tf.placeholder(tf.float32, shape=[None, num_classes])
keep = tf.placeholder(tf.float32)

In [5]:
weights = {
    'wc1' : tf.get_variable('wc1', [7, 7, 3, 96], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),    # 240
    'wc2' : tf.get_variable('wc2', [5, 5, 96, 256], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),   # 120
    'wc3' : tf.get_variable('wc3', [3, 3, 256, 512], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),   # 60
    'wc4' : tf.get_variable('wc4', [3, 3, 512, 512], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),  # 30
    'wc5' : tf.get_variable('wc5', [3, 3, 512, 512], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()), # 15
    'wd1' : tf.get_variable('wd1', [14*14*512, 4048], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),
    'wd2' : tf.get_variable('wd2', [4048, 4049], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer()),
    'wd3' : tf.get_variable('wd3', [4049, num_classes], dtype=tf.float32, 
                            initializer=tf.contrib.layers.xavier_initializer())
}

biases = {
    'bc1' : tf.Variable(tf.zeros([96]), trainable=False, dtype=tf.float32),
    'bc2' : tf.Variable(tf.zeros([256]), trainable=False, dtype=tf.float32),
    'bc3' : tf.Variable(tf.zeros([512]), trainable=False, dtype=tf.float32),
    'bc4' : tf.Variable(tf.zeros([512]), trainable=False, dtype=tf.float32),
    'bc5' : tf.Variable(tf.zeros([512]), trainable=False, dtype=tf.float32),
    'bd1' : tf.Variable(tf.zeros([4048]), trainable=False, dtype=tf.float32),
    'bd2' : tf.Variable(tf.zeros([4049]), trainable=False, dtype=tf.float32),
    'bd3' : tf.Variable(tf.zeros([num_classes]), trainable=False, dtype=tf.float32)
}

In [6]:
def model(x, w, b, k):
    
    with tf.device('/gpu:0'):
        # 480 x 480
        conv1 = tf.nn.conv2d(x, w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
        act1 = tf.nn.relu(conv1 + b['bc1'])
        pool1 = tf.nn.max_pool(act1, ksize=[1, 3, 3, 1], strides=[1, 3, 3, 1], padding='SAME')
    
        # 160 x 160
        conv2 = tf.nn.conv2d(pool1, w['wc2'], strides=[1, 1, 1, 1], padding='SAME')
        act2 = tf.nn.relu(conv2 + b['bc2'])
        pool2 = tf.nn.max_pool(act2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
        # 80 x 80
        conv3 = tf.nn.conv2d(pool2, w['wc3'], strides=[1, 1, 1, 1], padding='SAME')
        act3 = tf.nn.relu(conv3 + b['bc3'])
        pool3 = tf.nn.max_pool(act3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        # 40 x 40
        conv4 = tf.nn.conv2d(pool3, w['wc4'], strides=[1, 1, 1, 1], padding='SAME')
        act4 = tf.nn.relu(conv4 + b['bc4'])
    
    with tf.device('/gpu:1'):
        # 40 x 40
        conv5 = tf.nn.conv2d(act4, w['wc5'], strides=[1, 1, 1, 1], padding='SAME')
        act5 = tf.nn.relu(conv5 + b['bc5'])
        pool5 = tf.nn.max_pool(act5, ksize=[1, 3, 3, 1], strides=[1, 3, 3, 1], padding='SAME')
        print(pool5)
    
    #print(pool5.shape)
        flat = tf.reshape(pool5, [-1, 14*14*512])
    
        act6 = tf.nn.relu(tf.matmul(flat, w['wd1'])+b['bd1'])
        drop1 = tf.nn.dropout(act6, keep_prob=k)
    
        act7 = tf.nn.relu(tf.matmul(drop1, w['wd2'])+b['bd2'])
        drop2 = tf.nn.dropout(act7, keep_prob=k)
    
        output = tf.matmul(drop2, w['wd3'])+b['bd3']
    
    out = {
        'act1':act1,
        'act2':act2,
        'act3':act3,
        'act4':act4,
        'act5':act5,
        'act6':act6,
        'act7':act7,
        'output':output
        
    }
    
    
    return out
    

In [7]:
pred = model(X, weights, biases, keep)

Tensor("MaxPool_3:0", shape=(?, 14, 14, 512), dtype=float32, device=/device:GPU:1)


In [8]:
cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred['output'], labels=Y))
with tf.device('/gpu:0'):
    train = tf.train.AdamOptimizer(0.001).minimize(cost)

collect = tf.equal(tf.argmax(pred['output'], axis=1), tf.argmax(Y, axis=1))

accuracy = tf.reduce_mean(tf.cast(collect, tf.float32))

In [9]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
#sess=tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [10]:
epochs = 10
batch = 5



for epoch in range(epochs):
    
    total_batch = int(len(new)/batch)
    
    print("\n{0}/{1}".format(epoch+1, epochs))
    
    for i in range(total_batch):
        
        batch_idx = np.random.choice(len(new), batch, replace=False)
        
        batch_x = new[batch_idx]
        
        batch_y = train_lab[batch_idx]
        
        sess.run(train, feed_dict={X:batch_x, Y:batch_y, keep:0.7})
        
        if i%10 ==0:
            
            test_idx = np.random.choice(len(new), 17, replace=False)
            
            print("Cost : ", sess.run(cost, feed_dict={X:new[test_idx], Y:train_lab[test_idx], keep:1.}))
            
            print("Accuracy : ", sess.run(accuracy, feed_dict={X:new[test_idx], Y:train_lab[test_idx], keep:1.}))
            
#saver.save(sess, './checkpoint/')        
print("\n Training Done!")


1/10
Cost :  12025.2
Accuracy :  0.588235
Cost :  0.698378
Accuracy :  0.352941
Cost :  0.710121
Accuracy :  0.294118
Cost :  0.841579
Accuracy :  0.470588
Cost :  0.693777
Accuracy :  0.411765

2/10
Cost :  0.693318
Accuracy :  0.352941
Cost :  0.693083
Accuracy :  0.647059
Cost :  0.693121
Accuracy :  0.529412
Cost :  0.693174
Accuracy :  0.411765
Cost :  0.693153
Accuracy :  0.705882

3/10
Cost :  0.693162
Accuracy :  0.470588
Cost :  0.693142
Accuracy :  0.647059
Cost :  0.693147
Accuracy :  0.588235
Cost :  0.693145
Accuracy :  0.588235
Cost :  0.693147
Accuracy :  0.588235

4/10
Cost :  0.693146
Accuracy :  0.647059
Cost :  0.693147
Accuracy :  0.588235
Cost :  0.693147
Accuracy :  0.352941
Cost :  0.693147
Accuracy :  0.647059
Cost :  0.693147
Accuracy :  0.352941

5/10
Cost :  0.693142
Accuracy :  0.470588
Cost :  0.693147
Accuracy :  0.470588
Cost :  0.693143
Accuracy :  0.470588
Cost :  0.693147
Accuracy :  0.705882
Cost :  0.693147
Accuracy :  0.647059

6/10
Cost :  0.69314